In [2]:
import pandas as pd
import geopandas as gpd
import mytools

# 导入数据

In [ ]:
#导入空洞数据
kongd = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/3-拆分后的空洞/覆盖空19481个添加场景区分大中小微.data')

In [ ]:
kongd.head(1)

In [ ]:
#导入栅格数据
grid = pd.read_csv('G:/1-规划/评估/软采评估/2020年6月/6月评估程序-模块化/1-基础数据/软采数据/栅格/6月栅格-按照采样点倒序去重.csv',encoding='gbk')

In [ ]:
grid.head(1)

# 栅格生成点图

In [ ]:
dya,city,grid_id,leftbottom_lon,left_bottom_lat,righttop_lon,righttop_lat,rsrp_count,rsrp_weak,un_count,un_weak,te_count,te_weak=[
    'a.day', 'a.region_name', 'a.grid_id', 'a.grid_leftbottom_longitude','a.grid_leftbottom_latitude', 
    'a.grid_righttop_longitude' ,'a.grid_righttop_latitude','a.rsrp_sample_count', 'a.rsrp_weak', 
    'a.un_maxrsrp_sample_count',  'a.un_maxrsrp_weak_l2', 'a.te_maxrsrp_sample_count','a.te_maxrsrp_weak_l2']
lon,lat = ['lon','lat']

In [ ]:
data_t_use = grid[[dya,city,grid_id,leftbottom_lon,left_bottom_lat,righttop_lon,righttop_lat,rsrp_count,
                   rsrp_weak,un_count,un_weak,te_count,te_weak]]

In [ ]:
#添加lon和lat
data_t_use.loc[:,lon] = (data_t_use[leftbottom_lon]+data_t_use[righttop_lon])/2
data_t_use.loc[:,lat] = (data_t_use[left_bottom_lat]+data_t_use[righttop_lat])/2
data_t_use_v1 = data_t_use[[dya,city,grid_id,lon,lat,rsrp_count,rsrp_weak,un_count,un_weak,te_count,te_weak]]

In [ ]:
data_t_use_v1['fgl'] = 1 - data_t_use_v1[rsrp_weak] / data_t_use_v1[rsrp_count]

In [ ]:
grid_p = mytools.gisn.add_points(data_t_use_v1,lon,lat)

# 空洞匹配栅格

In [ ]:
kongd_result = gpd.sjoin(kongd,grid_p)

In [ ]:
kongd_result['fgl'] = 1 - kongd_result[rsrp_weak] / kongd_result[rsrp_count]

# 数据输出

In [ ]:
#导出匹配到栅格的空洞
kongd_grid = kongd_result.drop_duplicates('id')[['id']]
kongd_2 = kongd.merge(kongd_grid,how='inner',on='id')
kongd_2.to_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/4-相关图层输出/5栅格图层/匹配到栅格的{}个空洞.data'.format(kongd_2.shape[0]))

In [ ]:
#导出匹配到空洞的栅格
grid_2 = kongd_result.drop_duplicates('a.grid_id')[[ 'a.grid_id']]
grid_2_r = data_t_use_v1.merge(grid_2,how='inner',on='a.grid_id')
grid_2_r.to_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/4-相关图层输出/5栅格图层/空洞匹配到的{}个栅格.data'.format(grid_2_r.shape[0]))

In [ ]:
#导出匹配到空洞的栅格-弱覆盖
grid_2_r.loc[grid_2_r['fgl']<0.7,'栅格类型'] ='严重弱覆盖栅格'
kd_yzrfg = grid_2_r.loc[grid_2_r['fgl']<0.7]
kd_yzrfg.to_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/4-相关图层输出/5栅格图层/空洞匹配到的严重弱覆盖{}个栅格.data'.format(kd_yzrfg.shape[0]))

In [ ]:
#导出匹配到空洞的栅格-严重弱覆盖
grid_2_r.loc[(grid_2_r['fgl']<0.936),'栅格类型'] ='弱覆盖栅格'
kd_rfg = grid_2_r.loc[(grid_2_r['fgl']<0.936)]
kd_rfg.to_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/4-相关图层输出/5栅格图层/空洞匹配到的{}个弱覆盖栅格.data'.format(kd_rfg.shape[0]))

In [ ]:
print(grid_2_r.shape[0],kd_rfg.shape[0],kd_yzrfg.shape[0])

In [3]:
kongd = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/3-拆分后的空洞/覆盖空19481个添加场景区分大中小微.data')

In [4]:
tu = mytools.gisn.maps()

In [5]:
tu.head(1)

,city,type,geometry
0,十堰,Polygon,"POLYGON ((110.76119 31.83382, 110.76163 31.834..."


In [6]:
tu_area = mytools.gisn.add_area(tu)

In [8]:
tu_area['面积'].sum()

186881417014.07336

In [9]:
kongd.head(1)

,city,type,geometry,面积,lon,lat,区域类型,id,制式,distance,标准面积,空洞面积比例,空洞级别
0,十堰,Polygon,"POLYGON ((111.57467 32.59737, 111.57724 32.598...",118802.970802,111.577452,32.595834,农村,kd_1,FDD900,2200.0,1.520531e+07,0.007813,微


In [10]:
kongd.groupby('区域类型').agg({'面积':'sum'})

,面积
区域类型,
一般城区,2.541138e+08
主城区,7.564957e+07
乡镇,4.301427e+06
农村,2.855962e+10
县城,3.090842e+08
